In [1]:
import pandas as pd 
from utils import read_df, save_df_parquet, sample_series
import ast
import nltk

In [2]:
df: pd.DataFrame = read_df('dados_tokenizados.csv')

In [3]:
df.sample(3)

,id,canal,texto_original,temas,texto_limpo,tokens,tokens_clean
2268,3064,Audiência Regional - Perus,"Boa noite a todos, todas e todes. Eu sou Mari...","['Saúde', 'Habitação', 'Cultura']","boa noite a todos, todas e todes. eu sou mari...","['boa', 'noite', 'a', 'todos', ',', 'todas', '...","['boa', 'noite', 'todos', 'todas', 'todes', 'm..."
2148,2904,Participe+,Meta 43 - Revisar - Deveria ser feito corredor...,['Transportes e Mobilidade'],meta 43 - revisar - deveria ser feito corredor...,"['meta', '43', '-', 'revisar', '-', 'deveria',...","['revisar', 'deveria', 'feito', 'corredor', 'o..."
1977,2718,Participe+,PROPOSTA DO NOVO VIÁRIO DE ACESSO AO DISTRITO ...,['Infraestrutura Urbana e Obras'],proposta do novo viário de acesso ao distrito ...,"['proposta', 'do', 'novo', 'viário', 'de', 'ac...","['proposta', 'novo', 'viario', 'acesso', 'dist..."


Para construir a bag of words vou precisar apenas dos tokens e da classificacao.

Terei que fazer o explode na classificação - pois um mesmo texto pode ter mais de um tema

In [4]:
df = df[['tokens_clean', 'temas']]

In [5]:
df.dtypes

tokens_clean    object
temas           object
dtype: object

In [6]:
df.head()

,tokens_clean,temas
0,"['boa', 'tarde', 'todas', 'todos', 'nome', 'fr...",['Meio Ambiente']
1,"['oi', 'mario', 'sergio', 'bortolotto', 'favor...",['Urbanismo e Licenciamento']
2,"['entao', 'boa', 'tarde', 'todos', 'todas', 'c...","['Zeladoria Urbana e melhorias de bairro', 'De..."
3,"['obrigada', 'agradeco', 'oportunidade', 'faco...","['Zeladoria Urbana e melhorias de bairro', 'De..."
4,"['bom', 'dia', 'todos', 'nome', 'maria', 'luiz...",['Zeladoria Urbana e melhorias de bairro']


In [7]:
def parse_lst_column(lst_str)->list[str]:

    if type(lst_str) is not str:
        #bizarro - mas alguns registros estao como listas ou como arrays mesmo
        lst_str = f"[{', '.join(lst_str)}]"

    lst_str = lst_str.lstrip('array')

    return ast.literal_eval(lst_str)
    

In [8]:
df['temas'] = df['temas'].apply(parse_lst_column)
df['tokens_clean'] = df['tokens_clean'].apply(parse_lst_column)

In [9]:
df.sample(3)

,tokens_clean,temas
1740,"[piu, arco, tiete, elaborar, processo, partici...",[Urbanismo e Licenciamento]
188,"[oi, boa, noite, todos, prazer, ne, participar...",[Cultura]
1138,"[revitalizacao, parque, dom, pedro, implantaca...","[Urbanismo e Licenciamento, Infraestrutura Urb..."


In [10]:
df = df.explode('temas')
df.rename({'temas' : 'tema'}, axis=1, inplace=True)

df.sample(3)

,tokens_clean,tema
1421,"[regionalizar, orcamento, publico, municipal, ...",NaN
235,"[boa, noite, ketlyn, professora, regiao, migue...",Zeladoria Urbana e melhorias de bairro
77,"[bom, dia, todos, todas, vandoir, jose, souza,...",Habitação


In [11]:
df.reset_index(drop=True, inplace=True)

In [12]:
df['id'] = df.index

In [13]:
df.head()

,tokens_clean,tema,id
0,"[boa, tarde, todas, todos, nome, francisco, ap...",Meio Ambiente,0
1,"[oi, mario, sergio, bortolotto, favor, respond...",Urbanismo e Licenciamento,1
2,"[entao, boa, tarde, todos, todas, comprimento,...",Zeladoria Urbana e melhorias de bairro,2
3,"[entao, boa, tarde, todos, todas, comprimento,...",Desenvolvimento Econômico e Trabalho,3
4,"[obrigada, agradeco, oportunidade, faco, parte...",Zeladoria Urbana e melhorias de bairro,4


In [14]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
df['corpora'] = df['tokens_clean'].apply(lambda x: ' '.join(x))

In [16]:
df.head()

,tokens_clean,tema,id,corpora
0,"[boa, tarde, todas, todos, nome, francisco, ap...",Meio Ambiente,0,boa tarde todas todos nome francisco apelido c...
1,"[oi, mario, sergio, bortolotto, favor, respond...",Urbanismo e Licenciamento,1,oi mario sergio bortolotto favor responde enco...
2,"[entao, boa, tarde, todos, todas, comprimento,...",Zeladoria Urbana e melhorias de bairro,2,entao boa tarde todos todas comprimento secret...
3,"[entao, boa, tarde, todos, todas, comprimento,...",Desenvolvimento Econômico e Trabalho,3,entao boa tarde todos todas comprimento secret...
4,"[obrigada, agradeco, oportunidade, faco, parte...",Zeladoria Urbana e melhorias de bairro,4,obrigada agradeco oportunidade faco parte rede...


In [17]:
df = df[['id', 'corpora', 'tema']]

In [18]:
df.head()

,id,corpora,tema
0,0,boa tarde todas todos nome francisco apelido c...,Meio Ambiente
1,1,oi mario sergio bortolotto favor responde enco...,Urbanismo e Licenciamento
2,2,entao boa tarde todos todas comprimento secret...,Zeladoria Urbana e melhorias de bairro
3,3,entao boa tarde todos todas comprimento secret...,Desenvolvimento Econômico e Trabalho
4,4,obrigada agradeco oportunidade faco parte rede...,Zeladoria Urbana e melhorias de bairro


In [19]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import RSLPStemmer

# Baixar pacotes necessários do NLTK
nltk.download("rslp")

[nltk_data] Downloading package rslp to /home/h-pgy/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [20]:
# Criar Stemmer para português
stemmer = RSLPStemmer()

def stem_text(text):
    words = word_tokenize(text.lower())  # Tokeniza e converte para minúsculas
    return " ".join([stemmer.stem(word) for word in words if word.isalnum()])

In [21]:
df['corpora'] = df['corpora'].apply(stem_text)

In [22]:
sample_series(df['corpora'])

'boa noit tod gost paraben prefeit trabalh realiz plan met paraben pesso vere dou arnald far sa subprefeit tiag tod outr lid particip dess debat gost faz algum contribuico are saud gost indic program saud preven cad subprefeit jabaqu atend tod ub serv odontolog instal seil especi jabaqu part assistenc soc instal espac atend populaca vulnera situaca rua galpa pud aprend pesso aument capac restaurant bom prat atend pesso regia construca ceu jabaqu dev jabaqu ter mil habit implantaca escol integr tod escol municip instalaca curs profissionaliz red municip instalaca fab lab livr paul tod escol capacitaca geraca rend pesso gost indic cult criaca pont cult cad bairr jabaqu criaca centr memor jabaqu questa esport instalaca prac esport radic jabaqu instituica jog municip idos crianc mulh part mei ambi liberaca cooper recicl aument planti arv instalaca educaca ambient escol invest melh limp correg instal aument numer parqu line descentral tod secret subprefeit instalaca lamp led tod post publ r

precisa dropar as duplicadas - está confundindo o modelo uma mesma frase ter mais de uma classificacao

In [23]:
df = df.drop_duplicates('corpora')

In [24]:
vectorizer = CountVectorizer()

In [25]:
bow_matrix= vectorizer.fit_transform(df['corpora'])

In [26]:
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=vectorizer.get_feature_names_out())

In [27]:
bow_df.head()

,1000km,100km,10ampli,10h,10km,119c10201210,11o,12e3,12milho,12o,...,zhang,zild,zitu,zn,zogb,zom,zon,zone,zr1,zum
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
df.shape

(2212, 3)

In [29]:
bow_df.shape

(2212, 8647)

In [30]:
bow_df['id'] = df['id']

In [31]:
bow_df['y'] = df['tema']

In [32]:
bow_df.head()

,1000km,100km,10ampli,10h,10km,119c10201210,11o,12e3,12milho,12o,...,zild,zitu,zn,zogb,zom,zon,zone,zr1,zum,y
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Meio Ambiente
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Urbanismo e Licenciamento
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Zeladoria Urbana e melhorias de bairro
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Zeladoria Urbana e melhorias de bairro


In [33]:
save_df_parquet(bow_df, 'bag_of_words_df.parquet')

'../data/bag_of_words_df.parquet'